In [1]:
from frust.stepper import Stepper
from pathlib import Path
from tooltoad.chemutils import xyz2mol
import pandas as pd

In [2]:
structures_path = Path("../structures/test2_structs")

mols = {}

for f in structures_path.rglob("*.xyz"):
    with open(f, "r") as file:
        xyz_block = file.read()
        mol = xyz2mol(xyz_block)
        mols[f.stem] = (mol, [0])

In [3]:
mols

{'ts3_my_opt': (<rdkit.Chem.rdchem.Mol at 0x7fb9c72a2490>, [0])}

In [4]:
list(mols.keys())

['ts3_my_opt']

In [6]:
step = Stepper(list(mols.keys()), step_type="IRC", save_output_dir=False)

In [7]:
df0 = step.build_initial_df(mols)

In [8]:
df0

,custom_name,ligand_name,rpos,constraint_atoms,cid,smiles,atoms,coords_embedded,energy_uff
0,ts3_my_opt,my_opt,<NA>,None,0,None,"[H, C, C, C, C, H, C, H, C, H, B, H, H, H, H, ...","[(1.11908789810475, -4.6601126513316, 1.575671...",None


In [9]:
dfs = []
for i in range(len(df0)):
    df_tmp = df0.iloc[[i]]
    dfs.append(df_tmp)

def run_irc(df, debug=False, n_cores=12, mem_gb=30):
    from pathlib import Path
    name = df["custom_name"].iloc[0]

    step = Stepper([name],
                step_type="IRC",
                debug=debug,
                save_output_dir=False,
                n_cores=n_cores,
                memory_gb=mem_gb)
    
    detailed_inp = """%IRC
    MAXITER 40
    END
    """
    options = {
        "wB97X-D3" : None,
        "6-31G**"  : None,
        "TightSCF" : None,
        "Freq"     : None,
        "NoSym"    : None,
        "IRC"      : None,
    }

    df1 = step.orca(df, "DFT-IRC-my-opt", options, detailed_inp, save_step=True)
    results_dir = Path("results_irc")
    results_dir.mkdir(exist_ok=True)
    df1.to_parquet(f"{results_dir}/df_irc_{name}.parquet")

In [10]:
import submitit

DEBUG           = False
N_CORES         = 24
MEM_GB          = 60
TIMEOUT_MIN     = 14400


executor = submitit.AutoExecutor("logs/irc")
executor.update_parameters(
    slurm_partition="kemi1",
    cpus_per_task=N_CORES,
    mem_gb=MEM_GB,
    timeout_min=TIMEOUT_MIN,
)

for dfi in dfs:
    name = "IRC_" + dfi["custom_name"].iloc[0] 
    executor.update_parameters(slurm_job_name=name)
    executor.submit(run_irc, dfi, DEBUG, N_CORES, MEM_GB)
    print(f"Submitted: {name}")

Submitted: IRC_ts3_my_opt
